In [1]:
import sys
import os
sys.path.append("/mnt/c/Users/bach484/Documents/EcoliTRN-ModuleEval/src/modeval/")

import json
import shutil

from util import JSONExtendedEncoder

from clustering import *

import pandas as pd
import numpy as np

import time

import traceback
from modulecontainers import Modules

%load_ext autoreload
%autoreload 2

%matplotlib inline
from matplotlib.pyplot import *

import multiprocessing as mp

from collections import OrderedDict

from itertools import product
from collections import defaultdict

import itertools
import shutil

conf_folder = "conf/"

import os

from modulescomparison import ModevalKnownmodules, ModevalCoverage

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting git+https://github.com/jfrelinger/cython-munkres-wrapper

Cloning https://github.com/jfrelinger/cython-munkres-wrapper to /tmp/pip-req-build-epulhx5b

Running command git clone --filter=blob:none --quiet https://github.com/jfrelinger/cython-munkres-wrapper /tmp/pip-req-build-epulhx5b

Resolved https://github.com/jfrelinger/cython-munkres-wrapper to commit 38588d619373cfc7487dc92f323e46608550df2c

Preparing metadata (setup.py): started

Preparing metadata (setup.py): finished with status 'done'

In [2]:
settings = []
settings.append({
    "dataset_location":"conf/ecoli_dataset.json", 
    "dataset_name":"ecoli_precise2",
    "method_location":"conf/baseline_permuted.json", 
    "regnet_name":"ecoli_regulondb",
    "knownmodules_name":"minimal",
    "output_folder":"results/baseline_permuted/",
    "settingid": 1
    }
)

In [18]:
method_path = "../conf/baseline_permuted.json"
dataset_path = "../conf/ecoli_dataset.json"
output_folder = "../results/baseline_permuted/"

# Load the method and dataset JSON files
with open(method_path, "r") as file:
    method = json.load(file)

with open(dataset_path, "r") as file:
    dataset = json.load(file)

knownmodules = json.load(open(dataset["knownmodules"]["ecoli_regulondb"]["minimal"]))

starttime = time.time()

try:
	modules = locals()[method["params"]["method"]](knownmodules, **method["params"])
except BaseException as e:
	print("Error during clustering: ")
	print(e)
	traceback.print_exc()
	modules = []

endtime = time.time()

if os.path.exists(output_folder):
	shutil.rmtree(output_folder)
os.makedirs(output_folder)
json.dump(modules, open(output_folder + "modules.json", "w"), cls=JSONExtendedEncoder)
json.dump({"runningtime":endtime-starttime}, open(output_folder + "runinfo.json", "w"), cls=JSONExtendedEncoder)
json.dump(method["params"], open(output_folder + "method.json", "w"))


  Cloning https://github.com/jfrelinger/cython-munkres-wrapper to /tmp/pip-req-build-4nrdg4mp


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Running command git clone --filter=blob:none --quiet https://github.com/jfrelinger/cython-munkres-wrapper /tmp/pip-req-build-4nrdg4mp


  Resolved https://github.com/jfrelinger/cython-munkres-wrapper to commit 38588d619373cfc7487dc92f323e46608550df2c
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


In [19]:
# create pool of processors
if "pool" in locals().keys():
    pool.close()
pool = mp.Pool(mp.cpu_count()-1)

In [20]:
modeval = ModevalKnownmodules(settings, baseline = False)
modeval.run(pool)

In [26]:
scores = []
for setting in settings:
    score = modeval.scores.loc[
        (modeval.scores["settingid"] == setting["settingid"]) &
        (modeval.scores["knownmodules_name"] == setting["knownmodules_name"]) &
        (modeval.scores["regnet_name"] == setting["regnet_name"])
    ]
    
scores.append(score)
scores = pd.concat(scores)
scores["baseline_name"] = "permuted"
baseline = scores.groupby(["baseline_name", "regnet_name", "knownmodules_name"]).mean()

NumExpr defaulting to 8 threads.


In [27]:
scores

,recovery,relevance,F1rr,recall,precision,F1rp,F1rprr,settingid,knownmodules_name,regnet_name,goldstandard,runningtime,baseline_name
0,0.066481,0.067078,0.066778,0.018968,0.019032,0.019,0.029583,1,minimal,ecoli_regulondb,ecoli_regulondb#minimal,0.012276,permuted
